# PythonMonkey Examples
This notebook contains a number of example PythonMonkey examples. Play around with them get a feel for using PythonMonkey for JavaScript execution in Python.

In [30]:
!pip install --extra-index-url https://nightly.pythonmonkey.io/ --pre pythonmonkey


Looking in indexes: https://pypi.org/simple, https://nightly.pythonmonkey.io/


# Examples using pythonmonkey.eval to evaluate JavaScript code

In [ ]:
"""
console.log example

This example demonstrates printing to stdout from JS evaluated in Python.
"""
import pythonmonkey as pm

pm.eval("console.log('Hello from JavaScript');")

Hello from JavaScript


In [ ]:
"""
toUpperCase example

This example demonstrates executing a JavaScript function in Python which uses
a JavaScript string method toUpperCase.
"""
import pythonmonkey as pm

upify = pm.eval("(s) => s.toUpperCase()")
print(upify("Python string passed to a js function"))

PYTHON STRING PASSED TO A JS FUNCTION


In [54]:
"""
Python Dictionary to JavaScript Object example

This example demonstrates defining a function in JS that logs properties of a js
object and executing that function from Python with a Python Dictionary passed.
"""
import pythonmonkey as pm

my_dict = {
    "a": 1,
    "b": 2,
}

pm.eval("""
    function logObject(obj) {
        console.log(`a = ${obj.a}`);
        console.log(`b = ${obj.b}`);
    }
""")

pm.eval("logObject")(my_dict)

a = 1
b = 2


In [ ]:
"""
JavaScript Object to Python Dictionary-like example

This example demonstrates passing an object defined in JavaScript to Python.
"""

js_obj = pm.eval("({ a: 1, b: () => 2 })")

print(js_obj) # {'a': 1.0, 'b': <built-in method JSFunctionCallable of tuple object at 0x7b5d50926f80>}
print(js_obj.a) # 1.0
print(js_obj['a']) # 1.0
print(js_obj.b()) # 2.0

{'a': 1.0, 'b': <built-in method JSFunctionCallable of tuple object at 0x7b5d509311c0>}
1.0
1.0
2.0


# Examples using pythonmonkey.globalThis to get global variables from JS

In [ ]:
"""
Accessing JS api from globalThis

This example demonstrates a couple different ways to access JS APIs. This
example will show how you can use it to access the WebAssembly JS api from
JavaScript.
"""
import pythonmonkey
print(pythonmonkey.eval("WebAssembly")) # one method of getting the WASM api
print(pythonmonkey.globalThis.WebAssembly) # pm.globalThis can also be used

In [ ]:
"""
Accessing a global JavaScript variable defined in a context from Python.
"""
import pythonmonkey as pm

# define a global variable in a JavaScript context
pm.eval("myAwesomeGlobal = 7;")

# access it using globalThis or eval
print(pm.eval("myAwesomeGlobal")) # 7.0
print(pm.globalThis.myAwesomeGlobal) # 7.0

# modify the variable using globalThis api
pm.globalThis.myAwesomeGlobal = 22

# print to see if it changes to 22
print(pm.eval("myAwesomeGlobal")) # 22.0
print(pm.globalThis.myAwesomeGlobal) # 22.0

7.0
7.0
22.0
22.0


# Using JavaScript Promises in Python with asyncio
Note: we'll need to install nest_asyncio to get these examples working in a Google Colab environemnt. You will not need to use this library in typical Python environments.

In [ ]:
!pip install nest_asyncio

In [38]:
"""
Running a JavaScript Promise in Python

This example will demonstrate using asyncio to await a promise which resolves
after 2 seconds.
"""
import pythonmonkey
import asyncio

## !!!
# Since we're running this example inside of google colab, we need to nest the
# async event loop using nest_asyncio... You will not need to run the following
# two lines in typical cases.
import nest_asyncio
nest_asyncio.apply()
## !!!

async def someAsyncStuff():
    two_sec_promise = pythonmonkey.eval("""
        new Promise((resolve, reject) => {
            setTimeout(resolve, 2000);
        });
    """)

    print("before calling the 2 second timer")
    await two_sec_promise
    print("two seconds have now passed")

asyncio.run(someAsyncStuff())

before calling the 2 second timer
two seconds have now passed


In [ ]:
"""
Executing WebAssembly from Python

PythonMonkey uses SpiderMonkey, which is a JavaScript and WebAssembly engine.
Using the JavaScript WebAssembly API and calling it from Python using PythonMonkey,
we can execute WebAssembly easily in Python.

Refer to the MDN documentation for more information on WebAssembly.instantiate
https://developer.mozilla.org/en-US/docs/WebAssembly/JavaScript_interface/instantiate
"""
import pythonmonkey
import asyncio
WebAssembly = pythonmonkey.globalThis.WebAssembly

## !!!
# Since we're running this example inside of google colab, we need to nest the
# async event loop using nest_asyncio... You will not need to run the following
# two lines in typical cases.
import nest_asyncio
nest_asyncio.apply()
## !!!

# This is some WebAssembly code which calculates the factorial in a function called
# fac. Typically you would read a file in python and not hard code the bytes into
# the file like this.
wasm_bytes = bytearray(b'\x00asm\x01\x00\x00\x00\x01\x06\x01`\x01|\x01|\x03\x02\x01\x00\x07\x07\x01\x03fac\x00\x00\n.\x01,\x00 \x00D\x00\x00\x00\x00\x00\x00\xf0?c\x04|D\x00\x00\x00\x00\x00\x00\xf0?\x05 \x00 \x00D\x00\x00\x00\x00\x00\x00\xf0?\xa1\x10\x00\xa2\x0b\x0b\x00\x12\x04name\x01\x06\x01\x00\x03fac\x02\x03\x01\x00\x00')

async def getWasmExports(wasm_bytes):
    # WebAssembly.instantiate returns a promise which we'll await in python
    return (await WebAssembly.instantiate(wasm_bytes, {})).instance.exports

my_wasm_module = asyncio.run(getWasmExports(wasm_bytes))

# execute WebAssembly code in Python!
print(my_wasm_module.fac(4)) # this outputs "24.0" since factorial(4) == 24
print(my_wasm_module.fac(5)) # this outputs "120.0"
print(my_wasm_module.fac(6)) # this outputs "720.0"



24.0
120.0
720.0


# Example using pythonmonkey.require to load JavaScript modules in Python
I created a file located at /simple-math.js that contains the following code:


```
function add(a,b) {
  return a + b;
}

function sub(a,b) {
  return a - b;
}

module.exports = {
  add,
  sub,
}
```
We'll load this JavaScript module from Python and call its methods using `pythonmonkey.require`.


In [41]:
import pythonmonkey as pm
js_lib = pm.require('/simple-math')

print(js_lib.add(1,2)) #  3.0
print(js_lib.sub(1,2)) # -1.0

3.0
-1.0


# Example executing Python from within JavaScript
Using PythonMonkey, you can also execute Python code directly in your JavaScript code. Here are some examples demonstrating that.

In [42]:
import pythonmonkey as pm
pm.eval("""
    python.print("Hello from Python called from JavaScript called from Python");
""")

Hello from Python called from JavaScript called from Python


In [59]:
import pythonmonkey as pm
pm.eval("""
    // define a function in python
    python.exec(`
def pySquare(num):
  return num * num
    `);

    // execute the function defined in Python in js and print the answer
    console.log( python.eval('pySquare')(7) ); // 49
    console.log( python.eval('pySquare')(5) ); // 25
""")

49
25


# Example using pythonmonkey.new to instantiate JavaScript objects
Since Python lacks a 'new' keyword for instantiating objects, but some JavaScript objects require the usage of 'new', PythonMonkey provides an API for wrapping JavaScript constructors in new so they can be called from Python.

In [40]:
import pythonmonkey as pm

my_date = pm.new(pm.globalThis.Date)(0) # instantiates a new date object
print(my_date)

AttributeError: ignored